In [1]:
from keras.backend import in_test_phase
from tensorflow.python.ops.numpy_ops import np_config
from keras import backend as K
from scipy.spatial import distance
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


import gc
import csv
import ast
import random
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import os
import time

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
np_config.enable_numpy_behavior()

# The requirements to use the cuDNN implementation are:

# activation == tanh
# recurrent_activation == sigmoid
# recurrent_dropout == 0
# unroll is False
# use_bias is True
# Inputs, if use masking, are strictly right-padded.
# Eager execution is enabled in the outermost context.


def get_cross_train_rule_data_and_validation_data(dir):
    total_data_list= []
 
    session_list= []
    with open(dir, 'r+', newline='') as csvfile:    
        reader = csv.reader(csvfile, delimiter=',')
        row_count= 0
        for row in reader:
            row_list= []
            for col in row:
                if not col:
                    break
               
                if ast.literal_eval(col)== True:
                    data_list= ast.literal_eval(col) 
                    if not data_list in total_data_list:
                        row_list.append(data_list)
                else:
                    data_int= ast.literal_eval(col) 
                    if not data_int in total_data_list:
                        row_list.append(data_int)
            
            total_data_list.append(row_list)

            row_count+= 1
            if row_count> 16: # Select rows
                break
        

        num_of_session= 5    # Number of cross validation session
    
        for s in range(num_of_session):
            start_val_data= int(len(total_data_list)/ num_of_session* s)
            num_of_val_data= int(row_count* 1/ 3)
            train_rule_data_list= []
            validation_data_list= []
            new_data=  start_val_data+ num_of_val_data
            for row_index in range(len(total_data_list)):
                val_data_count= 0
                if row_index in range(start_val_data, start_val_data+ num_of_val_data):       
                    validation_data_list.append(total_data_list[row_index])
                else:
                    train_rule_data_list.append(total_data_list[row_index])
            session_list.append([train_rule_data_list, validation_data_list])

    return session_list

def get_rand_train_rule_data_and_validation_data(dir):
    total_data_list= []
 
    session_list= []
    with open(dir, 'r+', newline='') as csvfile:    
        reader = csv.reader(csvfile, delimiter=',')
        row_count= 0
        for row in reader:
            row_list= []
            for col in row:
                if not col:
                    break

                if ast.literal_eval(col)== True:
                    data_list= ast.literal_eval(col) 
                    if not data_list in total_data_list:
                        row_list.append(data_list)
                else:
                    data_int= ast.literal_eval(col) 
                    if not data_int in total_data_list:
                        row_list.append(data_int)
            
            total_data_list.append(row_list)

            row_count+= 1
            if row_count> 16: # Select rows
                break
        
        num_of_train_data= int(row_count* 2/ 3)    
        num_of_val_data= int(len(total_data_list)- num_of_train_data)
        session_num= 2    # cross validation

        
        rand_sel_record= []
        
        for session in range(session_num):
            total_data_list_copy= total_data_list.copy()
            validation_data_list= []
            for num in range(num_of_val_data):
                rand_sel= random.choice(total_data_list_copy)
                while rand_sel in rand_sel_record:
                    rand_sel= random.choice(total_data_list_copy)
                total_data_list_copy.remove(rand_sel)
                validation_data_list.append(rand_sel)
                rand_sel_record.append(rand_sel)
            train_rule_data_list= total_data_list_copy
            session_list.append([train_rule_data_list, validation_data_list])
    
    return session_list


    

def get_target_train_rule_data_and_validation_data_test_data(dir):
    total_data_list= []
    target_row= list(range(19, 24))  # 19- 24
    target_train_data_index_list= [25]
    target_val_data_for_histroy_record= [1, 3, 5, 7, 9, 10, 12]
    session_list= []
    specific_validation_data_list= []
    train_data_list= []
    with open(dir, 'r+', newline='' , encoding='utf-8-sig') as csvfile:    
        reader = csv.reader(csvfile, delimiter=',')
        row_count= 0
        
        for row in reader:
          
            row_list= []
            
            if len(row[2])== 0:
                continue
            for col in row:
                if not col:
                    break

                if ast.literal_eval(col)== True:
                    data_list= ast.literal_eval(col) 
                    if not data_list in total_data_list:
                        row_list.append(data_list)
                else:
                    data_int= ast.literal_eval(col) 
                    if not data_int in total_data_list:
                        row_list.append(data_int)
            
            total_data_list.append(row_list) 
            if row_count in target_row:
                specific_validation_data_list.append(row_list)
            else:
                train_data_list.append(row_list)

            row_count+= 1
        
        
        num_of_train_data= 30
        num_of_val_data_rate= 0.2
        num_of_val_data= int(num_of_train_data* num_of_val_data_rate)
        num_of_session= 3    # cross validation
       
       
        
        for session in range(num_of_session):
            validation_data_list= []
            train_data_list_copy= train_data_list.copy()
            number_of_val_count= 0
            
           
            if session== 0:
                while number_of_val_count< num_of_val_data:
                    rand_sel= random.choice(train_data_list_copy)
                    
                    validation_data_list.append(rand_sel)
                    number_of_val_count+= 1
                train_data_list_copy= train_data_list_copy[: num_of_train_data]
                #while len(train_data_list_copy)> num_of_train_data:
                #    rand_sel= random.choice(train_data_list_copy)
                #    train_data_index= train_data_list.index(rand_sel)
                #    if train_data_index in target_train_data_index_list:
                #        continue
                #    train_data_list_copy.remove(rand_sel)
                    
                target_val_data_list= validation_data_list
            if session== 1:
                target_val_data_list= specific_validation_data_list
            

            if session== 2:
                for i in target_val_data_for_histroy_record:
                    target_sel= train_data_list_copy[i]
                    
                   # specific_validation_data_list.remove(target_sel)
                    number_of_val_count+= 1
                    validation_data_list.append(target_sel)
                target_val_data_list= validation_data_list

          
                
            
            session_list.append([train_data_list_copy, target_val_data_list])

                
            print ('total data len', len(total_data_list))
            print ('train data len', len(train_data_list_copy))
            print ('validation data len', len(target_val_data_list))
            print ('specific_validation_data_list len', len(specific_validation_data_list))
  
    
    return session_list

def save_train_and_val_data(dir, data):
    with open(dir, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

def save_train_history_data(dir, data_list):
    with open(dir, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for data in data_list:
            writer.writerow(data)

def get_train_history_data(dir):
    with open(dir, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        data_list= [i for i in reader]
    return data_list

def get_train_and_val_data(dir):
    with open(dir, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        content= []
        for sub_content in reader:
            sub_content_list= []
            for s_s_content in sub_content:
                s_s_c= ast.literal_eval(s_s_content)
                sub_content_list.append(s_s_c)
            content.append(sub_content_list)
      
    return content





def get_layer_output_value(model, input_data_list_arr):
    inp = model.input
    outputs = [layer.output for layer in model.layers]   
    functors = [K.function([inp], [out]) for out in outputs] 
    layer_outs = [func([input_data_list_arr]) for func in functors]
    print ('layer_outs shape', np.array(layer_outs).shape)
   

  
def sub_cross_train_rule_data(train_rule_data):
    session_num= 5
    train_rule_data_len= len(train_rule_data)
    session_list= []

    for s in range(session_num):
        start_split_data_index= int(train_rule_data_len/ session_num* s)
        end_split_data_index= start_split_data_index+ train_rule_data_len-2
        if end_split_data_index> train_rule_data_len:
            end_split_data_index- train_rule_data_len
        if start_split_data_index< end_split_data_index:
            session_list.append(train_rule_data[start_split_data_index:end_split_data_index])
        else:
            session_list.append(train_rule_data[0:end_split_data_index]+ train_rule_data[start_split_data_index:-1])

    return session_list    
    

def sort_to_combination(sort_list):
    combination_list= []
    for sub_component_index in range(len(sort_list)):
        if sub_component_index== len(sort_list)- 1:
            break
        sub_component= [sort_list[sub_component_index], sort_list[sub_component_index+ 1]]
        combination_list.append(sub_component)
    return combination_list

def data_dist_info(component_data, train_rule_data_sort, expected_data_sort, outside_segment_data_sort):
    data_dist_list= []
    all_segments= expected_data_sort+ outside_segment_data_sort
    outside_segment_info= len(all_segments)+ 1
    component_position_in_train_rule_data_list= []
    component_position_in_expected_data_list= []
    component_position_in_outside_data_list= []
    if component_data in expected_data_sort:
        for segment_index in expected_data_sort:
      #  if not segment_index in train_rule_data_sort or not component_data in train_rule_data_sort: # The segments not in expected data. The dist== length of expected data + 1
      #      data_dist=  len(all_segments)+ 1
      #      data_dist_list.append(data_dist)
      #      continue
    
            component_data_index= train_rule_data_sort.index(component_data) # Component position
            expected_data_index= expected_data_sort.index(segment_index) # Each segments position
            data_dist= abs(expected_data_index- component_data_index) # The distance of component data position to expected data position. If the component data not in expected data dist info == 0.
            data_dist_list.append(data_dist) 
            component_position_in_train_rule_data_list.append(component_data_index)
            component_position_in_expected_data_list.append(expected_data_index)
        for outside_segment_index in outside_segment_data_sort:
            data_dist_list.append(outside_segment_info)
        
        dist_val= distance.minkowski(component_position_in_train_rule_data_list, component_position_in_expected_data_list, len(component_position_in_train_rule_data_list))
        dist_coding_list= []
        dist_val= list(str(dist_val))
        dist_val.remove('.')
        for dist in dist_val:
            dist_coding_list.append(int(dist))
        while len(dist_coding_list)< 16:
            dist_coding_list.append(0)
        while len(dist_coding_list)> 16:
            dist_coding_list.pop(-1)
        dist_coding_list= list(np.array(dist_coding_list)+ len(expected_data_sort)*2)

    else:
        for segment_index in expected_data_sort:
            data_dist_list.append(outside_segment_info)

        for outside_segment_index in outside_segment_data_sort:
            component_data_index= train_rule_data_sort.index(component_data)
            outside_data_index= outside_segment_data_sort.index(outside_segment_index)

            component_position_in_train_rule_data_list.append(component_data_index)
            component_position_in_outside_data_list.append(outside_data_index)

            data_dist= abs(outside_data_index- component_data_index)
            data_dist_list.append(data_dist) 
        
        dist_val= distance.minkowski(component_position_in_train_rule_data_list, component_position_in_outside_data_list, len(component_position_in_train_rule_data_list))
        dist_coding_list= []
        dist_val= list(str(dist_val))
        dist_val.remove('.')
        for dist in dist_val:
            dist_coding_list.append(int(dist))
        while len(dist_coding_list)< 16:
            dist_coding_list.append(0)
        while len(dist_coding_list)> 16:
            dist_coding_list.pop(-1)
        dist_coding_list= list(np.array(dist_coding_list)+ len(expected_data_sort)*2)

    return dist_coding_list


def data_conn_in_different_level_info(component_data, train_rule_data_sort, expected_data_sort, outside_segment_data_sort):
    data_conn_info_list= []
    all_segments= expected_data_sort+ outside_segment_data_sort
    component_data_index= train_rule_data_sort.index(component_data)
    conn= len(all_segments)+ 7
    not_conn= len(all_segments)+ 8
    

   
    still_conn= True
        
    for next_conn_in_order in range(1, len(expected_data_sort)):
        conn_level= not_conn+ next_conn_in_order
        data_conn_info_list.append(conn_level)        
        if component_data in expected_data_sort:
            component_data_categ= expected_data_sort.index(component_data)
            component_data_in_train_data_categ= train_rule_data_sort.index(component_data)     
            

            conn_info= None
            
            if component_data_categ== len(expected_data_sort):
                conn_info= not_conn
                still_conn= False
            
                
            else:    
                conn_categ_train_data=  component_data_in_train_data_categ+ next_conn_in_order     
                conn_categ= component_data_categ+ next_conn_in_order

                if conn_categ_train_data>= len(train_rule_data_sort):
                   
                    conn_info= not_conn
                
              
                if conn_categ>= len(train_rule_data_sort):
                    conn_info= not_conn               
                
              
                if conn_categ_train_data< len(train_rule_data_sort) and conn_categ< len(train_rule_data_sort):
                    if train_rule_data_sort[conn_categ_train_data] ==  expected_data_sort[conn_categ]:
                        if still_conn:
                            conn_info= conn
                        else:
                            conn_info= not_conn
                    else:
                        conn_info= not_conn
               

        else: 
            component_data_categ= outside_segment_data_sort.index(component_data)
            outside_component_data_categ= len(expected_data_sort)+ component_data_categ
              
            conn_info= not_conn

        data_conn_info_list.append(conn_info)
    
    return data_conn_info_list


def data_conn_info(component_data, train_rule_data_sort, expected_data_sort, outside_segment_data_sort):
    data_conn_info_list= []
    all_segments= expected_data_sort+ outside_segment_data_sort
    component_data_index= train_rule_data_sort.index(component_data)
    conn= len(all_segments)+ 1
    not_conn= len(all_segments)+ 2
    
    
    for segment in train_rule_data_sort: 
        conn_info= None
        if segment in expected_data_sort:
            segment_categ= expected_data_sort.index(segment)
            data_conn_info_list.append(segment_categ)
           
            if segment_categ== len(expected_data_sort):
                conn_info= not_conn
               
                
            else:
                conn_categ_train_data= segment+ 1
                conn_categ= segment_categ+ 1
                
                if conn_categ_train_data== len(train_rule_data_sort):
                    conn_info= not_conn
                if conn_categ== len(train_rule_data_sort):
                    conn_info= not_conn
                    
                if conn_categ_train_data!= len(train_rule_data_sort) and conn_categ!= len(train_rule_data_sort):
                    if train_rule_data_sort[conn_categ_train_data] ==  expected_data_sort[conn_categ]:
                        conn_info= conn
                    else:
                        conn_info= not_conn
        
        else: 
            segment_categ= outside_segment_data_sort.index(segment)
            outside_component_data_categ= len(expected_data_sort)+ segment_categ
            data_conn_info_list.append(outside_component_data_categ)          
            conn_info= not_conn
      

        data_conn_info_list.append(conn_info)
            
    
    if component_data in expected_data_sort:
        conn_info= None
        component_data_categ= expected_data_sort.index(component_data)
        component_data_in_train_data_categ= train_rule_data_sort.index(component_data)
        data_conn_info_list.append(component_data_categ)
        if component_data_categ== len(expected_data_sort):
            conn_info= not_conn
            
                
        else:    
            conn_categ_train_data=  component_data_in_train_data_categ+ 1       
            conn_categ= component_data_categ+ 1

            if conn_categ_train_data== len(train_rule_data_sort):
                conn_info= not_conn
                
              
            if conn_categ== len(train_rule_data_sort):
                conn_info= not_conn               
                
              
            if conn_categ_train_data!= len(train_rule_data_sort) and conn_categ!= len(train_rule_data_sort):
                if train_rule_data_sort[conn_categ_train_data] ==  expected_data_sort[conn_categ]:
                    conn_info= conn
                else:
                    conn_info= not_conn
               
            

    else: 
        component_data_categ= outside_segment_data_sort.index(component_data)
        outside_component_data_categ= len(expected_data_sort)+ component_data_categ
        data_conn_info_list.append(outside_component_data_categ)          
        conn_info= not_conn

    data_conn_info_list.append(conn_info)
    
    return data_conn_info_list



    

def all_segments_data_posi_info(train_rule_data_sort, expected_data_sort, outside_segment_data_sort):
    all_segments_len= len(expected_data_sort+ outside_segment_data_sort)
    r_posi= all_segments_len+ 3
    not_r_posi= all_segments_len+ 4
    all_segments_data_posi_info_list= []
    for i, segment in enumerate(train_rule_data_sort):
        posi_cate= i+ len(expected_data_sort)
        posi_info= None
        if not segment in expected_data_sort:
            
            posi_info= not_r_posi
            all_segments_data_posi_info_list.append(posi_cate)
            all_segments_data_posi_info_list.append(posi_info)
            continue 
           
        component_data_index= train_rule_data_sort.index(segment)
        expected_data_index= expected_data_sort.index(segment)
        if component_data_index== expected_data_index:
            posi_info= r_posi
        else:
            posi_info= not_r_posi
        all_segments_data_posi_info_list.append(posi_cate)
        all_segments_data_posi_info_list.append(posi_info)

    return all_segments_data_posi_info_list



def data_posi_info(component_data, train_rule_data_sort, expected_data_sort, outside_segment_data_sort):
    all_segment_len= len(expected_data_sort+ outside_segment_data_sort)
    all_segment= expected_data_sort+ outside_segment_data_sort
   
    r_posi= all_segment_len+ 5
    not_r_posi= all_segment_len+ 6
    posi_info= None

    if not component_data in expected_data_sort:
        
        posi_info= not_r_posi
    else:
        component_data_index= train_rule_data_sort.index(component_data)
        expected_data_index= expected_data_sort.index(component_data)
        if component_data_index== expected_data_index:
            posi_info= r_posi
        else:
            posi_info= not_r_posi

    return posi_info
    
def get_activity_as_represntation(data_list):
    number_of_units= len(data_list)
    data_list_np= np.expand_dims(np.array(data_list), 0)
    output= tf.keras.layers.Dense(units = number_of_units, activation='sigmoid', use_bias=True)(data_list_np)
    output= output.numpy().tolist()
    return output

def in_right_dataset_info(component_data, train_rule_data_sort, expected_data_sort, outside_segment_data_sort):
    all_segment_len= len(expected_data_sort+ outside_segment_data_sort)
   
    in_right_d= all_segment_len+ 7
    not_in_right_d= all_segment_len+ 8
    in_right_info= not_in_right_d

    if not component_data in expected_data_sort:
        in_right_info= not_in_right_d
    else:
       
        in_right_info= in_right_d
    
    return in_right_info


def get_input_data(expected_data_sort, outside_segment_data_sort, train_rule_data_list, validation_data_list):
    input_data_list= [] # [ data numbers* component numbers- 1, 2, component numbers of expected data -1]
    input_data_index_list= []
    validation_input_data_list= []
    validation_data_input_index_list= []

  
    for train_rule_data in train_rule_data_list:            
        train_rule_data_sort= train_rule_data[0]
        train_rule_data_comb_list= sort_to_combination(train_rule_data_sort)
        for train_rule_data_comb in train_rule_data_comb_list:
            first_component= train_rule_data_comb[0]
            second_component= train_rule_data_comb[1]
         #   first_comp_data_dist_info_train_list= data_dist_info(first_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
         #   second_comp_data_dist_info_train_list= data_dist_info(second_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
         #   data_conn_in_different_level_list= data_conn_in_different_level_info(first_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
            first_comp_data_conn_info_train_list= data_conn_info(first_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
         #   second_comp_data_conn_info_train_list= data_conn_info(second_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
            all_segments_data_posi_info_list= all_segments_data_posi_info(train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
            
            first_comp_data_posi_info= data_posi_info(first_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
            second_comp_data_posi_info= data_posi_info(second_component, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
            
       
            
            input_data= first_comp_data_conn_info_train_list+ all_segments_data_posi_info_list+ [first_comp_data_posi_info]+ [second_comp_data_posi_info]
           

          #  activity_as_representation= get_activity_as_represntation(input_data)
            input_data_prepro= on_preprocessing_layer(input_data, len(input_data))
           
            input_data= input_data_prepro          
            input_data_list.append(input_data)
      

            input_data_index_list.append([train_rule_data_sort, [first_component, second_component]])

    for validation_data in validation_data_list: 
        validation_data_sort= validation_data[0]
        validation_data_comb_list= sort_to_combination(validation_data_sort)
        for validation_data_comb in validation_data_comb_list:
            val_data_first_component= validation_data_comb[0]
            val_data_second_component= validation_data_comb[1]
            
          #  first_comp_data_dist_info_val_list= data_dist_info(val_data_first_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)
          #  second_comp_data_dist_info_val_list= data_dist_info(val_data_second_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)

         #   all_data_conn_info_list= all_data_conn_info(val_data_first_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)
          #  val_data_conn_in_different_level_list= data_conn_in_different_level_info(val_data_first_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)
            first_comp_data_conn_info_val_list= data_conn_info(val_data_first_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)
          #  second_comp_data_conn_info_val_list= data_conn_info(val_data_second_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)
            all_segments_data_posi_info_list= all_segments_data_posi_info(train_rule_data_sort, expected_data_sort, outside_segment_data_sort)

            first_comp_data_posi_info= data_posi_info(val_data_first_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)
            second_comp_data_posi_info= data_posi_info(val_data_second_component, validation_data_sort, expected_data_sort, outside_segment_data_sort)

           
            val_input_data= first_comp_data_conn_info_val_list+ all_segments_data_posi_info_list+ [first_comp_data_posi_info]+ [second_comp_data_posi_info]
           

         #   activity_as_representation= get_activity_as_represntation(val_input_data)  
            val_input_data_prepro= on_preprocessing_layer(val_input_data, len(val_input_data))
            val_input_data= val_input_data_prepro
            validation_input_data_list.append(val_input_data)

            validation_data_input_index_list.append([validation_data_sort, [val_data_first_component, val_data_second_component]])
  
    return input_data_list, input_data_index_list, validation_input_data_list, validation_data_input_index_list

def get_input_data_posi_val(expected_data_sort, outside_segment_data_sort, train_rule_data_list, validation_data_list):
    input_data_list= [] # [ data numbers* component numbers- 1, 2, component numbers of expected data -1]
    input_data_index_list= []
    validation_input_data_list= []
    validation_data_input_index_list= []
    for train_rule_data in train_rule_data_list:            
        train_rule_data_sort= train_rule_data[0]      
        for train_rule_data_comb in train_rule_data_sort:  
            comp_data_conn_info_list= data_conn_info(train_rule_data_comb, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
            comp_data_posi_info= data_posi_info(train_rule_data_comb, train_rule_data_sort, expected_data_sort, outside_segment_data_sort)
         
            input_data= comp_data_conn_info_list
            input_data.append(comp_data_posi_info)

            input_data_list.append(input_data)
      

            input_data_index_list.append([train_rule_data_sort, train_rule_data_comb])
    for validation_data in validation_data_list:            
        validation_data_sort= validation_data[0]       
        for val_data_comb in validation_data_sort:
            comp_data_conn_info_list= data_conn_info(val_data_comb, validation_data_sort, expected_data_sort, outside_segment_data_sort)
            comp_data_posi_info= data_posi_info(val_data_comb, validation_data_sort, expected_data_sort, outside_segment_data_sort)

            val_input_data= comp_data_conn_info_list
            val_input_data.append(comp_data_posi_info)

            validation_input_data_list.append(val_input_data)
            validation_data_input_index_list.append([validation_data_sort, val_data_comb])
  
    return input_data_list, input_data_index_list, validation_input_data_list, validation_data_input_index_list

def get_input_data_0(expected_data_sort, train_rule_data_list, validation_data_list):
    input_data_list= [] # [ data numbers* component numbers- 1, 2, component numbers of expected data -1]
    input_data_index_list= []
    validation_input_data_list= []
    validation_data_input_index_list= []
    for train_rule_data in train_rule_data_list:            
        train_rule_data_sort= train_rule_data[0]
        train_rule_data_comb_list= sort_to_combination(train_rule_data_sort)
        for train_rule_data_comb in train_rule_data_comb_list:
            first_component= train_rule_data_comb[0]
            second_component= train_rule_data_comb[1]
        
            first_comp_data_dist_info_list= data_dist_info(first_component, train_rule_data_sort, expected_data_sort)
            second_comp_data_dist_info_list= data_dist_info(second_component, train_rule_data_sort, expected_data_sort)
            first_comp_data_posi_info= data_posi_info(first_component, train_rule_data_sort, expected_data_sort)
            second_comp_data_posi_info= data_posi_info(second_component, train_rule_data_sort, expected_data_sort)
     
  
            input_data= train_rule_data_sort+ train_rule_data_comb+ first_comp_data_dist_info_list+ second_comp_data_dist_info_list
            input_data.append(first_comp_data_posi_info)
            input_data.append(second_comp_data_posi_info)
            input_data_list.append(input_data)
      

            input_data_index_list.append([train_rule_data_sort, [first_component, second_component]])
    for validation_data in validation_data_list:            
        validation_data_sort= validation_data[0]
        validation_data_comb_list= sort_to_combination(validation_data_sort)
        for validation_data_comb in validation_data_comb_list:
            val_data_first_component= validation_data_comb[0]
            val_data_second_component= validation_data_comb[1]

            first_comp_data_dist_info_list= data_dist_info(val_data_first_component, train_rule_data_sort, expected_data_sort)
            second_comp_data_dist_info_list= data_dist_info(val_data_second_component, train_rule_data_sort, expected_data_sort)
            first_comp_data_posi_info= data_posi_info(val_data_first_component, train_rule_data_sort, expected_data_sort)
            second_comp_data_posi_info= data_posi_info(val_data_second_component, train_rule_data_sort, expected_data_sort)

    
            val_input_data= validation_data_sort+ validation_data_comb+ first_comp_data_dist_info_list+ second_comp_data_dist_info_list
            val_input_data.append(first_comp_data_posi_info)
            val_input_data.append(second_comp_data_posi_info)
        

            validation_input_data_list.append(val_input_data)
            validation_data_input_index_list.append([validation_data_sort, [val_data_first_component, val_data_second_component]])
  
    return input_data_list, input_data_index_list, validation_input_data_list, validation_data_input_index_list



def get_expected_output_data(input_data_index_list, train_rule_data_list, output_list):
    all_output_val_list= []
    train_rule_data_index_list= [i[0] for i in train_rule_data_list]
    for input_data_index in input_data_index_list:
        target_train_rule_data_index= train_rule_data_index_list.index(input_data_index[0])
        target_train_rule_data= train_rule_data_list[target_train_rule_data_index]
        for component_index in range(len(target_train_rule_data)):
            if component_index== 0:
                continue
          
            if component_index% 2== 1 and type(target_train_rule_data[component_index])== list:
                if input_data_index[1] == [target_train_rule_data[component_index][0], target_train_rule_data[component_index][2]]:                     
                    target_output= target_train_rule_data[component_index+ 1]
                    output_index_list= [i[0] for i in output_list]
                    target_output_index= output_index_list.index(str(target_output))
                    output_val_list= [i[1] for i in output_list]
                    output_val_list[target_output_index]= 1
        all_output_val_list.append(output_val_list)
        
    return all_output_val_list
        

def get_expected_output_data_posi_val(input_data_index_list, train_rule_data_list, output_list):
    all_output_val_list= [] 
    train_rule_data_index_list= [i[0] for i in train_rule_data_list]
    for input_data_index in input_data_index_list:
        target_train_rule_data_index= train_rule_data_index_list.index(input_data_index[0])
        target_train_rule_data= train_rule_data_list[target_train_rule_data_index]
        for component_index in range(len(target_train_rule_data)):
            if component_index== 0:
                continue
            if component_index% 2== 1 and type(target_train_rule_data[component_index])== int:
                if input_data_index[1] == target_train_rule_data[component_index]:
                    target_output= target_train_rule_data[component_index+ 1]
          
                    output_index_list= [i[0] for i in output_list]
                    target_output_index= output_index_list.index(str(target_output))
                    output_val_list= [i[1] for i in output_list]
                    output_val_list[target_output_index]= 1
        all_output_val_list.append(output_val_list)

    return all_output_val_list


def get_expected_output_data_all_data_level(input_data_index_list, train_rule_data_list, output_list):
    all_output_list= []
  
    train_rule_data_index_list= [i[0] for i in train_rule_data_list]
    for input_data_index in range(len(input_data_index_list)):
    
        output_val_list= list(np.zeros(len(input_data_index_list)))
        output_val_list[input_data_index]= 1
        all_output_list.append(output_val_list)
    return all_output_list

def get_current_result(session_list, session_index, expected_data_sort, outside_segment_data_sort, output_list, save_model_dir):
    train_rule_data_list= session_list[session_index][0]
    validation_data_list= session_list[session_index][1]
        
    input_data_list, input_data_index_list, validation_input_data_list, validation_data_input_index_list= get_input_data(expected_data_sort, outside_segment_data_sort, train_rule_data_list, validation_data_list)
   
    model= load_model(save_model_dir)
    sse, mse= init_predict_on_training(model, expected_data_sort, validation_input_data_list, validation_data_input_index_list, validation_data_list, output_list)

    return sse

def dropout_non_connect(model):
    target_layer_id_list= [1, 2, 3, 4, 5, 6]
    for layer_id in target_layer_id_list:

                target_lstm_layer_weights = model.layers[layer_id].get_weights()[0]
                target_lstm_layer_weights_shape= target_lstm_layer_weights.shape
                target_lstm_layer_weights_size= target_lstm_layer_weights.size
                new_target_lstm_layer_weights= target_lstm_layer_weights.reshape(target_lstm_layer_weights_size)
                
                for weight in new_target_lstm_layer_weights:
                    if weight< 0:
                        weight= 0
                  
                new_target_lstm_layer_weights= new_target_lstm_layer_weights.reshape(target_lstm_layer_weights_shape)
                new_param=  model.layers[layer_id].get_weights()
                new_param[0]= new_target_lstm_layer_weights           
                model.layers[layer_id].set_weights(new_param)
                
    return model

def output_as_feedback_signal(model):
   # get_last_layer_output = K.function([model.layers[0].input],
   #                               [model.layers[-1].output])
   # last_layer_output = get_last_layer_output([])[0]
    last_layer_output= model.layers[-1].output
    print (last_layer_output.get_shape())
    print (last_layer_output)
    target_layer_id_list= [1, 2, 3, 4, 5, 6]
    for layer_id in target_layer_id_list:

                target_lstm_layer_weights = model.layers[layer_id].get_weights()[0]
                target_lstm_layer_weights_shape= target_lstm_layer_weights.shape
                target_lstm_layer_weights_size= target_lstm_layer_weights.size
                new_target_lstm_layer_weights= target_lstm_layer_weights.reshape(target_lstm_layer_weights_size)
                
                for weight in new_target_lstm_layer_weights:
                    if weight< 0:
                        weight= 0
                  
                new_target_lstm_layer_weights= new_target_lstm_layer_weights.reshape(target_lstm_layer_weights_shape)
                new_param=  model.layers[layer_id].get_weights()
                new_param[0]= new_target_lstm_layer_weights           
                model.layers[layer_id].set_weights(new_param)
                
    return model

    
    



def init_training(expected_data_sort, outside_segment_data_sort, session_list, output_list, num_of_layers, model, memory_state, carry_state, save_model_dir, checkpoint_dir, save_train_history_dir, train_rule_data_dir):
    init_train= True
    init_next_layer= False
    save_model_dir=  save_model_dir
    checkpoint_dir= checkpoint_dir 
    

    seed= 10
    tf.keras.utils.set_random_seed(seed)

    state= (memory_state, carry_state)
    layer_count= 0
    session_index= 0
    num_of_layers= num_of_layers
    train_times_count= 0
    call_back_stop_count= 0
    call_back_stop_val= 5
    pred_result_list= []
 
    reset_train_history_data= True
    if os.path.exists(save_train_history_dir) or reset_train_history_data== False:
        result_history_list= get_train_history_data(save_train_history_dir)
    if reset_train_history_data:
        result_history_list= []
     # Get input, output data
    train_rule_data_list= session_list[session_index][0]
    validation_data_list= session_list[session_index][1]
    # train_rule_data_list= sub_cross_train_rule_data(train_rule_data_list)[sub_session_index]
        

       
    input_data_list, input_data_index_list, validation_input_data_list, validation_data_input_index_list= get_input_data(expected_data_sort, outside_segment_data_sort, train_rule_data_list, validation_data_list)

    output_value_list= get_expected_output_data(input_data_index_list, train_rule_data_list, output_list)  # Output expeceted data    
   


  
   

    if model:
        current_result= get_current_result(session_list, session_index, expected_data_sort, outside_segment_data_sort, output_list, save_model_dir)
        pred_result_list.append(current_result)

    while init_train:
        gc.enable()
        gc.collect()
        print ('previous pred_result_list', pred_result_list)
        
        print ('length of train data', len(train_rule_data_list))
        print ('length of validation data', len(validation_data_list))
        
   
        input_data_list_arr= np.array(input_data_list)  # Input data
        output_value_list_arr= np.array(output_value_list)  # Output data


     
        batch_size, time_steps, features= input_data_list_arr.shape
        print ('batch', batch_size, 'time_steps', time_steps, 'feature', features)
        
        

      #  if model:
      #      model= dropout_non_connect(model)
         

        if not model:      
            #####  Build model  #####   
            input_layer= on_input_layer(input_data_list_arr, batch_size, time_steps, features) # For encoder, decoder

            layer= input_layer
            init_next_layer= True
            while init_next_layer:
                if layer_count== num_of_layers:
                    break
                new_layer, memory_state, carry_state= on_next_layer(layer, layer_count, num_of_layers, state, time_steps, features)
                state= (memory_state, carry_state)
                layer= new_layer
                layer_count+= 1

      
            output_layer= on_output_layer(input_data_list_arr, output_value_list_arr, layer, time_steps, features)  

            model = tf.keras.Model(inputs= input_layer, outputs= output_layer, name= 'reward_predict_model') # Multi
            

            #  model= build_model()
  

    
        #### Batch  training
        tf.keras.backend.clear_session()
        history= train_model(input_data_list_arr, output_value_list_arr, model, checkpoint_dir, batch_size)
        model_c= tf.keras.models.clone_model(
            model, input_tensors=None, clone_function=None
        )
       

        val_accuracy_list= history.history['val_accuracy']
        accuracy_list= history.history['accuracy']


        get_layer_output_value(model, input_data_list_arr)

        init_pred= False
        
        if train_times_count% 5== 0 or init_pred:
            if max(np.array(accuracy_list))>=0.9:
                tf.keras.backend.clear_session()
                sse, mse= init_predict_on_training(model_c, expected_data_sort, validation_input_data_list, validation_data_input_index_list, validation_data_list, output_list) 
                print ('sse :', sse)
                pred_result_list.append(sse)
                best_result= min(pred_result_list)
                if len(pred_result_list)> 1 and sse== best_result:
                    save_model(model, save_model_dir)
                    print ('Save the best result :', sse)

                    call_back_stop_count= 0
                call_back_stop_count+= 1

        if call_back_stop_count== call_back_stop_val:                   
            break


        if train_times_count% 20== 0:
            if not max(np.array(accuracy_list))>=0.9:
                save_model(model, save_model_dir)
        
        if train_times_count% 5== 0:
            session_index= 1

            train_rule_data_list_for_hist= session_list[session_index][0]
            validation_data_list_for_hist= session_list[session_index][1]
            predict_data_list_for_hist= validation_data_list

            input_data_list_for_hist, input_data_index_list_for_hist, val_input_data_list_for_hist, val_data_input_index_list_for_hist= get_input_data(expected_data_sort, outside_segment_data_sort, train_rule_data_list_for_hist, validation_data_list_for_hist)

            print (len(train_rule_data_list_for_hist), train_rule_data_list_for_hist)
            print (len(predict_data_list_for_hist), predict_data_list_for_hist)

            sse, mse= init_predict_on_training(model_c, expected_data_sort, val_input_data_list_for_hist, val_data_input_index_list_for_hist, validation_data_list_for_hist, output_list) 
               
            result_history_list.append([train_times_count, sse, mse])

            

            
            
       
       #     pred_result_list.append(sse)
       #     best_result= min(pred_result_list)
       #     if len(pred_result_list)> 1 and sse== best_result:
       #        save_model(model, save_model_dir)
       #        print ('Save the best result :', sse)
        #       call_back_stop_count= 0
        #    call_back_stop_count+= 1
   
        print (result_history_list)
        train_times_count+= 1
        save_train_history_data(save_train_history_dir, result_history_list)  
          
 
      
    return model, validation_input_data_list, validation_data_input_index_list, input_data_list



def on_preprocessing_layer_multi_hot(input_data_list, features):
    layer= tf.keras.layers.CategoryEncoding(num_tokens= features, output_mode="multi_hot")
    new_input_data_list= layer(input_data_list)
    #new_input_data_list= tf.keras.layers.Embedding( # Input_dim: Integer. i.e. maximum integer index + 1.
    #  1000,
    #  len(input_data_list),
    #  embeddings_initializer="uniform",
    #  embeddings_regularizer=None,
    #  activity_regularizer=None,
    #  embeddings_constraint=None,
    #  mask_zero=False,
    #  input_length=None,
    #)(input_data_list)
    return new_input_data_list


def on_preprocessing_layer(input_data_list, data_size):
    #data_range= len(input_data_list)#+ 6
    layer= tf.keras.layers.CategoryEncoding(num_tokens= data_size, output_mode="one_hot")
    new_input_data_list= layer(input_data_list)
    #new_input_data_list= tf.keras.layers.Embedding( # Input_dim: Integer. i.e. maximum integer index + 1.
    #  1000,
    #  len(input_data_list),
    #  embeddings_initializer="uniform",
    #  embeddings_regularizer=None,
    #  activity_regularizer=None,
    #  embeddings_constraint=None,
    #  mask_zero=False,
    #  input_length= None,
    #)(new_input_data_list)
  
    return new_input_data_list

def on_input_layer(input_data_list, batch_size, time_steps, features):
   # input= [samples, time_steps, features] [Number of datas, The length of each data, Each element of the data is a vector of n features]
   # Samples - Number of datas
   # Time steps -   The length of each data
   # Features - Each element of the data is a vector of n features
    input_layer = tf.keras.Input(shape=(time_steps, features), name='input_layer') # shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors.
    return input_layer
  
def on_next_layer(layer, layer_count, num_of_layers, state, time_steps, features):
    if state[0]== None:
        state= None
    if layer_count== num_of_layers-1:
        dropout= 0
    else:
        dropout= 0
    number_of_units= 256

  # if layer_count %2== 0:
  #   number_of_units= 14
  #   state= None   
  #   new_layer, memory_state, carry_state= tf.keras.layers.LSTM(units = number_of_units, name= 'hidden_layer_'+ str(layer_count), return_sequences=True, stateful= True, return_state= True
  #                                 , time_major= True, activation="selu",
  #   recurrent_activation="selu", unit_forget_bias=True,
  #   use_bias=True)(layer, state)
  # else:
  #   number_of_units= 14-1
  #   new_layer, _, _= tf.keras.layers.LSTM(units = number_of_units, name= 'hidden_layer_'+ str(layer_count), return_sequences=True, stateful= True, return_state= True
  #                                 , time_major= True, activation="selu",
  #   recurrent_activation="selu", unit_forget_bias=True,
  #   use_bias=True)(layer)
  #
  #   carry_state= None
  #   memory_state= None
   
    lstm= tf.keras.layers.LSTM(units = number_of_units, input_shape= (time_steps, features), name= 'hidden_layer_'+ str(layer_count), return_sequences=True, stateful= False, return_state= True
                                  , time_major= False, activation="tanh", recurrent_activation="sigmoid", unit_forget_bias=True, 
                                  kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", dropout= dropout, 
                                  use_bias=True)
  
    new_layer, memory_state, carry_state= lstm(layer, state)
    
    num_heads= int(math.sqrt(number_of_units))
 
    if layer_count== num_of_layers-1:
       
       
        new_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(features))(new_layer) # TimeDistributed applies the same instance of previous layer to each of the timestamps, the same set of weights are used at each timestamp.#

    return new_layer, memory_state, carry_state



def on_output_layer(intput_data_list, output_value_list, layer, time_steps, features):
 
    outputs= tf.keras.layers.Flatten(data_format=None)(layer)
    outputs= tf.keras.layers.Dropout(0.4) (outputs, training=True)
    outputs= tf.keras.layers.Dense(units = len(intput_data_list[0]), activation='relu', name='output_dense_layer_0', use_bias=True)(outputs)
    outputs= tf.keras.layers.Dense(units = len(intput_data_list[0]), activation='sigmoid', name='output_dense_layer_1', use_bias=True)(outputs)
    outputs= tf.keras.layers.Dense(units = len(output_value_list[0]), activation='relu', name='output_dense_layer_2', use_bias=True)(outputs)
    outputs= tf.keras.layers.Dense(units = len(output_value_list[0]), activation='sigmoid', name='output_dense_layer_3', use_bias=True)(outputs)
    print ('outputs shape', outputs.shape)

    return outputs

def train_model(input_data_list, data_expected_output, model, checkpoint_dir, batch_size):
    optimizer_Adam= tf.keras.optimizers.Adam(learning_rate= 1e-3,
                                              beta_1=0.9,
                                              beta_2=0.999,
                                              epsilon=1e-07,
    )

    rms_prop= tf.keras.optimizers.RMSprop(
        learning_rate=1e-3,
        rho=0.9,
        momentum=0.0,
        epsilon=1e-07,
        centered=False,
        name="RMSprop",
    )

    ada_delta= tf.keras.optimizers.Adadelta(
        learning_rate=1e-3, rho=0.95, epsilon=1e-07, name="Adadelta"
    )

    sgd= tf.keras.optimizers.SGD(learning_rate=1e-3)

    binary_crossentropy= tf.keras.losses.BinaryCrossentropy(
                from_logits= True,
                label_smoothing=0.0,
                axis=-1,
                reduction="auto",
                name="binary_crossentropy",
    )

    categorical_crossentropy= tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0.0,
        axis=-1,
      #  reduction="auto",
        name="categorical_crossentropy",
    ) 
    early_stopping= tf.keras.callbacks.EarlyStopping(
            # Stop training when `val_loss` is no longer improving
            monitor="val_loss", # val_loss
            # "No longer improving" being defined as "no better than 1e-2 less"
            min_delta=1e-2,
            # "No longer improving" being further defined as "for at least 2 epochs"
            patience=2,
            verbose=1,
            mode= 'auto'
    )


    callbacks = [
        early_stopping,
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_dir,
            monitor='val_accuracy',
            mode='auto',
            save_weights_only= True,
            save_best_only=True
        )    
    ]
 

    model.compile(#optimizer= sgd,
              optimizer= optimizer_Adam, 
              # Loss function to minimize
              loss= tf.keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor    
              metrics=['accuracy', 'mse']
            )

    print ('input_data_list shape', input_data_list.shape)
    print ('data_expected_output shape', data_expected_output.shape)
  
 
    print('# Fit model on training data')
    model.summary()
    batch_size= 32
    number_of_epochs= 50

    history = model.fit(input_data_list, 
            data_expected_output, 
        batch_size= batch_size, 
        epochs= number_of_epochs,
        validation_split=0.2,
        callbacks= callbacks,
                )
  
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

 # results = model.evaluate(validation_input_data_list, validation_expected_output, batch_size=128)
 # print("test loss, test acc:", results)
    return history

def build_model(batch_size, timesteps, features):
    batch_size= 3
    units= 512
    features= 5
    output_size= 3
    timesteps= 5
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    input_layer = tf.keras.layers.Input(shape=(timesteps, features))
        # The LSTM layer with default options uses CuDNN.
    lstm_layer= tf.keras.layers.LSTM(units, input_shape=(timesteps, features)
                                  , activation="tanh",
    recurrent_activation="sigmoid", unit_forget_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal",
    use_bias=True, batch_size= batch_size)
   


    return model





def save_model(Model, dir):
    Model.save(
        dir,
      overwrite=True,
      include_optimizer=True,
      save_format=None,
      signatures=None,
      options=None,
      save_traces=True,
    )

def load_model(dir):
    loaded_model = tf.keras.models.load_model(dir)
    return loaded_model

def model_predict(predict_data, model):
  
    prediction= model.predict(
        predict_data,
        batch_size=None,
        verbose="auto",
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
      )
    return prediction

def init_predict_on_training(model, all_segments, predict_input_data_list, predict_input_data_index, predict_data_list, output_list): 

    predict_input_data_list_arr= np.array(predict_input_data_list) 

    expected_output_value_list= get_expected_output_data(predict_input_data_index, predict_data_list, output_list)
    output_score_list= [i[0] for i in output_list]
  
    count= 1
    SSE= 0
    for predict_data, predict_data_index, expected_output_value_index in zip(predict_input_data_list_arr, predict_input_data_index, expected_output_value_list):
        detail= None
        SE= 0
        time_steps, features= predict_data.shape
        new_predict_data= predict_data.reshape(1, time_steps, features)
        prediction= model_predict(new_predict_data, model)
 
        max_val= np.max(prediction)
        test, value_index= np.where(prediction== max_val)
        if len(value_index)!= 1:
            print (predict_data_index, "can't predict data")
            continue
        predict_score= output_score_list[int(value_index)]
        new_expected_output_value_index= expected_output_value_index.index(1)
        expected_output_value= output_score_list[new_expected_output_value_index]
  
        SE+= (float(predict_score)- float(expected_output_value))** 2
        if SE!= 0:
            detail= ['predict_score', predict_score, 'expected_score', expected_output_value]
        SSE+= SE
   #   print (predict_data_index, '(predict score- data score)^ 2, SE: ',SE, 'detail: ', detail)
      
   #   if count% 6 == 0:
   #     print ()
    # print ('predict_score', predict_score )
        count+= 1
    MSE= SSE/ count
    return SSE, MSE




def get_ans_info_list(input_ans_list, expected_output, output_steps_records_list): 
    correct_ans_info_list= []
    ans_match_list= []
    for ans in input_ans_list[0]:
        if ans in expected_output:
            correct_ans_info= [0, ans[0], ans[1]]
            correct_ans_info_list.append(correct_ans_info)
        
        if not ans in expected_output: 
            ans_info= [1, ans[0], ans[1]]
            ans_match_list.append(ans_info)

    step_probability_list= []
    for i in output_steps_records_list:
        for i_0 in i[2]:
            all_search_in_step= True
            for c in range(len(ans_match_list)):             
                if ans_match_list[c] not in i_0:
                    all_search_in_step= False
                else:
                    if not [i[0], i[1],i_0] in step_probability_list:
                        ans_match_list[c].append([i[0], i[1], i_0])
                        step_probability_list.append([i[0], i[1], i_0])
        
            
    return ans_match_list, step_probability_list

def get_probility_score(full_score, ans_match_list, step_probability_list, input_ans_index_name):
    total_score= 0
    
    input_ans_list_2_upload= [] # Input ans 2 upload

    for math_ans_info in ans_match_list:
        #print ('math_ans_info', math_ans_info)
        match_ans_probability= math_ans_info[3][1]
        match_ans_info_list= math_ans_info[3][2]

        match_ans_score=  1/ int(match_ans_probability)* full_score
        
        print (math_ans_info[1], '->', math_ans_info[2], 'wrong answer: get score '+ str(round(match_ans_score, 1)), 'probability '+ str(1)+ '/', int(match_ans_probability))
        total_score+= match_ans_score

        # Ans info to dataset [src_id, target_id, src_label, target_label, description]
        src_id= math_ans_info[1]
        target_id= math_ans_info[2]
        arrow= 'to' # attribute 'from', 'to
        src_label= input_ans_index_name[math_ans_info[1]]
        description= 'wrong answer: get score '+ str(round(match_ans_score, 1))
        target_label= input_ans_index_name[math_ans_info[2]]
        color= 'rgb(255, 0, 0)'
        input_ans_list_2_upload.append([src_id, target_id, src_label, target_label, description, color]) 

    #print ('total_score', total_score)
    return total_score, input_ans_list_2_upload

def get_output_steps_records_list(dir):
    with open(dir, 'r', newline='') as csvfile:
        content= []
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            sub_content= []
            sub_content.append(row[0])
            sub_content.append(int(row[1]))
            step_record= ast.literal_eval(row[2])
            sub_content.append(step_record)      
            content.append(sub_content)
      
    return content




train_rule_data_name= 'train_rule_data_for_con_demo_man.csv'
output_steps_records_data_name= 'output_steps_records_list.csv'
train_and_val_data_name= 'train_data_for_val.csv'

dir= os.getcwd()
train_rule_data_dir= os.path.join(dir, train_rule_data_name)
save_train_and_val_data_dir= os.path.join(dir, train_and_val_data_name)

number_of_segments= 6
number_of_outside_number_of_segments= 6

outside_segment_data_sort= list(range(101, 100+ number_of_outside_number_of_segments+ 1))
expected_data_sort= list(range(1, number_of_segments+ 1))
all_segments_data_sort= expected_data_sort+ outside_segment_data_sort

number_of_layers= 6

output_list= [['-5', 0], ['-4', 0], ['-3', 0], ['-2', 0], ['-1', 0], ['0', 0], ['1', 0], ['2', 0], ['3', 0], ['4', 0], ['5', 0]]
#output_list= []
#for score in range(-5, 6):
#    sub_component= [str(score), 0]
#    output_list.append(output_list)

gen_new_session_list= True
if not os.path.exists(save_train_and_val_data_dir) or gen_new_session_list: 
    session_list= get_target_train_rule_data_and_validation_data_test_data(train_rule_data_dir) 
    save_train_and_val_data(save_train_and_val_data_dir, session_list)


session_list= get_train_and_val_data(save_train_and_val_data_dir)
#session_list= get_cross_train_rule_data_and_validation_data(target_dir)


memory_state= None
carry_state= None

task_name= '_for_conference'#'_for_conference' #_for_conference_with_test_model
save_model_dir= os.path.join(dir, 'eva_reward_predict_model_'+ str(number_of_layers)+'_layers'+ task_name) 
checkpoint_dir= os.path.join(save_model_dir, 'checkpoint/')
save_train_history_dir= os.path.join(dir, 'train_history.csv')

if not os.path.exists(save_model_dir):
    os.mkdir(save_model_dir)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

#output_steps_records_list_target_dir= os.path.join(dir, output_steps_records_data_name)
#output_steps_records_list= get_output_steps_records_list(output_steps_records_list_target_dir) # Get output content
#model= load_model(save_model_dir)
model= False
#model, predict_input_data_list, predict_input_data_index, input_data_list= init_training(expected_data_sort, outside_segment_data_sort, session_list, output_list, number_of_layers, model, memory_state, carry_state, save_model_dir, checkpoint_dir, save_train_history_dir, train_rule_data_dir)


#weights = model.get_weights() # Getting params
#model.set_weights(weights) # Setting params
#weights = model.layers[i].get_weights() # Getting params
#model.layers[i].set_weights(weights) # Setting params

total data len 51
train data len 30
validation data len 6
specific_validation_data_list len 5
total data len 51
train data len 46
validation data len 5
specific_validation_data_list len 5
total data len 51
train data len 46
validation data len 7
specific_validation_data_list len 5


In [2]:
import os

dir= os.getcwd()
save_model_dir= os.path.join(dir, 'eva_reward_predict_model_'+ str(number_of_layers)+ '_layers'+ task_name)

model= load_model(save_model_dir)
model.summary()

def load_model(dir):
    loaded_model = tf.keras.models.load_model(dir)
    return loaded_model

def model_predict(predict_data, model):
 
  prediction= model.predict(
    predict_data,
    batch_size=None,
    verbose="auto",
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
  )
  return prediction

def save_predict_data(target_dir, data):
    with open(target_dir, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)


def save_score_list(target_dir, data):
    with open(target_dir, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)



def init_predict(dir, model, all_segments, predict_input_data_list, predict_input_data_index, predict_data_list, output_list): 

  
    predict_input_data_list_arr= np.array(predict_input_data_list) 

    expected_output_value_list= get_expected_output_data(predict_input_data_index, predict_data_list, output_list)
    output_score_list= [i[0] for i in output_list]

    count= 1
    SSE= 0
    save_predict_data_list= []
    total_score= 0
    total_score_list= []
    for predict_data, predict_data_index, expected_output_value_index in zip(predict_input_data_list_arr, predict_input_data_index, expected_output_value_list):
        detail= 'Match'
        SE= 0
        time_steps, features= predict_data.shape
        new_predict_data= predict_data.reshape(1, time_steps, features)
        prediction= model_predict(new_predict_data, model)
 
        max_val= np.max(prediction)
        test, value_index= np.where(prediction== max_val)
        if len(value_index)!= 1:
            print (predict_data_index, "can't predict data")
            continue
        predict_score= output_score_list[int(value_index)]
        new_expected_output_value_index= expected_output_value_index.index(1)
        expected_output_value= output_score_list[new_expected_output_value_index]
  
        SE+= (float(predict_score)- float(expected_output_value))** 2
        if SE!= 0:
            detail= ['predict_score', predict_score, 'expected_score', expected_output_value]
        else:
            detail= ['Match  ', 'predict_score', predict_score, 'expected_score', expected_output_value]
        SSE+= SE
        print (predict_data_index, '(predict score- data score)^ 2, SE: ',SE, 'detail: ', detail)
        data_row= [predict_data_index, '(predict score- data score)^ 2, SE: ',SE, 'detail: ', detail]
        save_predict_data_list.append(data_row)
        total_score+= float(predict_score)
        if count% (int(len(predict_data_list[0][0]))- 1) == 0:
            save_predict_data_list.append([])
            total_score_list.append([predict_data_index[0], total_score])
            total_score= 0
            print ()
    # print ('predict_score', predict_score )
        count+= 1
    print (count)
    MSE= SSE/ count
    print ('SSE', SSE)
    print ('MSE', MSE)

    
    predict_data_dir= os.path.join(dir, 'predict_data.csv')
    predict_score_data_dir= os.path.join(dir, 'predict_score_data.csv')
    save_predict_data(predict_data_dir, save_predict_data_list)
    save_score_list(predict_score_data_dir, total_score_list)
    

    return SSE

    
session_index= 1

train_rule_data_list= session_list[session_index][0]
validation_data_list= session_list[session_index][1]
predict_data_list= validation_data_list

input_data_list, input_data_index_list, val_input_data_list, val_data_input_index_list= get_input_data(expected_data_sort, outside_segment_data_sort, train_rule_data_list, validation_data_list)



init_predict(dir, model, expected_data_sort, val_input_data_list, val_data_input_index_list, predict_data_list, output_list)


Model: "reward_predict_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 28, 28)]     0           []                               
                                                                                                  
 hidden_layer_0 (LSTM)          [(None, 28, 256),    291840      ['input_layer[0][0]']            
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                                  
 hidden_layer_1 (LSTM)          [(None, 28, 256),    525312      ['hidden_layer_0[0][0]',         
                                 (None, 256),                     'hidden_layer

1/1 [==============================] - 6s 6s/step
[[1, 2, 4, 5, 3, 106], [1, 2]] (predict score- data score)^ 2, SE:  0.0 detail:  ['Match  ', 'predict_score', '5', 'expected_score', '5']
1/1 [==============================] - 0s 32ms/step
[[1, 2, 4, 5, 3, 106], [2, 4]] (predict score- data score)^ 2, SE:  4.0 detail:  ['predict_score', '-5', 'expected_score', '-3']
1/1 [==============================] - 0s 35ms/step
[[1, 2, 4, 5, 3, 106], [4, 5]] (predict score- data score)^ 2, SE:  0.0 detail:  ['Match  ', 'predict_score', '3', 'expected_score', '3']
1/1 [==============================] - 0s 32ms/step
[[1, 2, 4, 5, 3, 106], [5, 3]] (predict score- data score)^ 2, SE:  0.0 detail:  ['Match  ', 'predict_score', '-5', 'expected_score', '-5']
1/1 [==============================] - 0s 30ms/step
[[1, 2, 4, 5, 3, 106], [3, 106]] (predict score- data score)^ 2, SE:  0.0 detail:  ['Match  ', 'predict_score', '-5', 'expected_score', '-5']

1/1 [==============================] - 0s 31ms/step
[[

158.0

In [ ]:
#number_of_trainin_data= 20
#SSE 104.0
#MSE 4.0
# history_list= [[0, 328.0, 15.619047619047619], [5, 385.0, 18.333333333333332], [10, 789.0, 37.57142857142857], [15, 494.0, 23.523809523809526], [20, 519.0, 24.714285714285715], [25, 617.0, 29.38095238095238], [30, 545.0, 25.952380952380953], [35, 297.0, 14.142857142857142], [40, 801.0, 38.142857142857146], [45, 459.0, 21.857142857142858], [50, 997.0, 47.476190476190474]]

In [ ]:

import csv
import ast
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import os
import time
a= list(range(1, 13))
print (a)

rand_list= []
for i in range(5):
  c_a= a.copy()
  random.shuffle(c_a)
  rand_list.append(c_a)

for i in rand_list:
  rand_2_dict= dict(enumerate(i))

  print (rand_2_dict)
  print (rand_2_dict[0])

dir= '/content/drive/MyDrive/Colab_drive/'
save_rand_data_dir= os.path.join(dir, 'rand_data.csv')
with open(save_rand_data_dir, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerows(rand_list)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
{0: 11, 1: 3, 2: 12, 3: 8, 4: 2, 5: 4, 6: 7, 7: 1, 8: 9, 9: 6, 10: 5, 11: 10}
11
{0: 9, 1: 3, 2: 10, 3: 12, 4: 1, 5: 11, 6: 5, 7: 2, 8: 7, 9: 4, 10: 8, 11: 6}
9
{0: 6, 1: 8, 2: 1, 3: 9, 4: 4, 5: 11, 6: 5, 7: 10, 8: 2, 9: 12, 10: 7, 11: 3}
6
{0: 10, 1: 6, 2: 12, 3: 5, 4: 2, 5: 7, 6: 8, 7: 9, 8: 3, 9: 11, 10: 4, 11: 1}
10
{0: 4, 1: 11, 2: 9, 3: 10, 4: 3, 5: 8, 6: 2, 7: 12, 8: 6, 9: 1, 10: 5, 11: 7}
4


In [ ]:
dir= '/content/drive/MyDrive/Colab_drive/'
save_rand_data_dir= os.path.join(dir, 'rand_data.csv')
with open(save_rand_data_dir, 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    rand_list= [i for i in reader]
     

for i in rand_list:
    rand_2_dict= dict(enumerate(i))
    print (i)
    print (rand_2_dict)
    

['4', '8', '7', '12', '1', '6', '2', '3', '11', '5', '10', '9']
{0: '4', 1: '8', 2: '7', 3: '12', 4: '1', 5: '6', 6: '2', 7: '3', 8: '11', 9: '5', 10: '10', 11: '9'}
['2', '9', '7', '5', '3', '1', '6', '12', '8', '11', '10', '4']
{0: '2', 1: '9', 2: '7', 3: '5', 4: '3', 5: '1', 6: '6', 7: '12', 8: '8', 9: '11', 10: '10', 11: '4'}
['12', '8', '11', '3', '9', '4', '5', '10', '2', '6', '7', '1']
{0: '12', 1: '8', 2: '11', 3: '3', 4: '9', 5: '4', 6: '5', 7: '10', 8: '2', 9: '6', 10: '7', 11: '1'}
['7', '2', '8', '12', '11', '3', '1', '4', '6', '5', '9', '10']
{0: '7', 1: '2', 2: '8', 3: '12', 4: '11', 5: '3', 6: '1', 7: '4', 8: '6', 9: '5', 10: '9', 11: '10'}
['2', '8', '3', '5', '6', '9', '12', '10', '4', '1', '7', '11']
{0: '2', 1: '8', 2: '3', 3: '5', 4: '6', 5: '9', 6: '12', 7: '10', 8: '4', 9: '1', 10: '7', 11: '11'}


In [ ]:
#number_of_trainin_data= 20
#SSE 104.0
#MSE 4.0
#history_list=[[0, 894.0, 24.833333333333332], [5, 930.0, 25.833333333333332], [10, 625.0, 17.36111111111111], [15, 969.0, 26.916666666666668], [20, 881.0, 24.47222222222222], [25, 559.0, 15.527777777777779], [30, 838.0, 23.27777777777778], [35, 1025.0, 28.47222222222222], [40, 926.0, 25.72222222222222]]


#number_of_trainin_data= 30
#MSE= 7.038
#history_list= [[0, 894.0, 24.833333333333332], [5, 930.0, 25.833333333333332], [10, 625.0, 17.36111111111111], [15, 989.0, 27.47222222222222], [20, 616.0, 17.11111111111111], [25, 642.0, 17.833333333333332], [30, 1200.0, 33.333333333333336], [35, 822.0, 22.833333333333332], [40, 654.0, 18.166666666666668]]

#number_of_data= 40
#hitory_list= [[0, 894.0, 24.833333333333332], [5, 930.0, 25.833333333333332], [10, 625.0, 17.36111111111111], [15, 989.0, 27.47222222222222], [20, 694.0, 19.27777777777778], [25, 593.0, 16.47222222222222], [30, 881.0, 24.47222222222222], [35, 1012.0, 28.11111111111111], [40, 928.0, 25.77777777777778]]

# number_of_data= 50
#hitory_list= [[0, 894.0, 24.833333333333332], [5, 930.0, 25.833333333333332], [10, 625.0, 17.36111111111111], [15, 989.0, 27.47222222222222], [20, 694.0, 19.27777777777778], [25, 614.0, 17.055555555555557], [30, 913.0, 25.36111111111111], [35, 948.0, 26.333333333333332], [40, 711.0, 19.75]]
